In [1]:
import datetime as dt

import numpy as np
import pandas as pd

In [2]:
raw_fundamental = pd.read_csv("./array_optimize.csv", index_col=0)

In [3]:
fundamental = raw_fundamental.copy()

In [4]:
fundamental["stock_code"] = fundamental["stock_code"].apply(lambda x: str(x).zfill(6))
fundamental.sample(5)

,date,stock_code,factor,amount,reprt_no
38571,2022-03-21,036640,current_assets,6.376410e+10,1
27224,2022-03-09,005090,total_assets,2.591550e+12,1
3835,2021-03-16,016580,raw_profit,1.717054e+11,2
68923,2023-03-22,003580,fixed_liabilities,2.890728e+09,0
26783,2022-03-01,018310,current_assets,NaN,2


In [5]:
def preprocessor(df, reprt_no):
    selected_df = df[df["reprt_no"] == reprt_no]
    df_pivot = selected_df.pivot(
        index=["stock_code", "date"], columns=["factor"], values=["amount"]
    )
    df_pivot.columns = df_pivot.columns.get_level_values(1)
    df_pivot = df_pivot.reset_index(level=1)
    df_pivot = df_pivot.loc[
        :,
        [
            "date",
            "current_assets",
            "current_liabilities",
            "total_assets",
            "total_liabilities",
            "net_profit",
            "operation_profit",
            "raw_profit",
            "price",
            "shares",
        ],
    ]
    df_pivot.dropna(axis=0, inplace=True)
    return df_pivot


fundamental_0 = preprocessor(fundamental,0)
fundamental_1 = preprocessor(fundamental,1)
fundamental_2 = preprocessor(fundamental,2)

In [6]:
# 가장 최근 연간재무제표
fundamental_0.head(3)

factor,date,current_assets,current_liabilities,total_assets,total_liabilities,net_profit,operation_profit,raw_profit,price,shares
stock_code,,,,,,,,,,
000020,2023-03-15,2.275275e+11,7.556234e+10,4.611365e+11,8.246321e+10,2.159121e+10,2.991508e+10,3.404263e+11,9172.18750,27931470.0
000040,2023-03-22,6.074121e+10,9.356145e+10,1.491852e+11,1.091612e+11,-1.300128e+10,-6.234393e+09,1.170587e+11,508.34375,96138465.0
000050,2023-03-17,1.498824e+11,1.995375e+11,1.248318e+12,4.933099e+11,4.415250e+09,3.189531e+10,3.892276e+11,10921.56250,27415270.0


In [7]:
# 가장 최근-1 연간재무제표
fundamental_1.head(3)

factor,date,current_assets,current_liabilities,total_assets,total_liabilities,net_profit,operation_profit,raw_profit,price,shares
stock_code,,,,,,,,,,
000020,2022-03-15,2.201986e+11,6.646130e+10,4.478045e+11,8.767127e+10,1.958280e+10,2.249635e+10,2.930181e+11,12493.75,27931470.0
000040,2022-03-22,6.966286e+10,9.552199e+10,1.684368e+11,1.182112e+11,-1.192456e+10,-2.697618e+09,1.335104e+11,801.25,96138465.0
000050,2022-03-17,1.255011e+11,2.307164e+11,1.270805e+12,5.186587e+11,2.228188e+10,5.379682e+10,3.833571e+11,14512.50,27415270.0


In [8]:
# 가장 최근-2 연간재무제표
fundamental_2.head(3)

factor,date,current_assets,current_liabilities,total_assets,total_liabilities,net_profit,operation_profit,raw_profit,price,shares
stock_code,,,,,,,,,,
000020,2021-03-15,2.227435e+11,6.749400e+10,4.338078e+11,9.112556e+10,2.871510e+10,2.315337e+10,2.720754e+11,15021.8750,27931470.0
000040,2021-03-22,6.097059e+10,8.412293e+10,1.558735e+11,1.060284e+11,-1.148601e+10,1.219024e+09,1.178344e+11,1251.0625,96138465.0
000050,2021-03-17,1.173308e+11,2.856408e+11,1.264902e+12,5.229722e+11,1.302570e+10,2.185400e+10,3.292247e+11,12565.6250,27415270.0


In [9]:
"""
여기서 claim
    - 일반적으로 LOW-PER / LOW-PBR 같은게 저평가 되어있다고 하는데
    - 역사적으로 수익률이 좋았던 지표는 어떤것들이 있었는가
    - 이게 보면 ML 모델 말고 그냥 지표의 quantile에 따른 해석을 하고싶을때가 있다.
"""

""

''

In [10]:
raw_ohlcv = pd.read_csv("./ohlcvs.csv", index_col=0, low_memory=False)

In [11]:
ohlcv = raw_ohlcv.copy()
ohlcv["stock_code"] = ohlcv["stock_code"].apply(lambda x: str(x).zfill(6))

In [12]:
ma_ohlcv = ohlcv.groupby("stock_code").rolling(window=20).mean().reset_index().dropna()
ma_ohlcv["date"] = pd.to_datetime(ma_ohlcv["date"])
ma_ohlcv.sort_values("date", inplace=True)

In [13]:
def merge_with_price(fundamental, ma_ohlcv):
    fundamental = fundamental.copy()
    fundamental.reset_index(inplace=True)
    fundamental["date"] = pd.to_datetime(fundamental["date"])
    fundamental["future_date"] = fundamental["date"] + dt.timedelta(days=60)
    fundamental.sort_values("future_date", inplace=True)
    final_df = pd.merge_asof(
        left=fundamental,
        right=ma_ohlcv.loc[:, ["stock_code", "date", "close"]],
        by=["stock_code"],
        left_on=["future_date"],
        right_on=["date"],
    )
    final_df = final_df.drop(columns=["date_y"]).rename(columns={"date_x": "date"})
    final_df["profit"] = (final_df["close"] - final_df["price"]) / final_df["price"]
    return final_df

In [14]:
final_0 = merge_with_price(fundamental_0, ma_ohlcv)
final_1 = merge_with_price(fundamental_1, ma_ohlcv)
final_2 = merge_with_price(fundamental_2, ma_ohlcv)

In [15]:
def append_factors(final_df):
    final_df["cbps"] = (final_df["current_assets"] - final_df["current_liabilities"]) / final_df[
        "shares"
    ]
    final_df["cpbr"] = final_df["cbps"] / final_df["price"]

    final_df["tbps"] = (final_df["total_assets"] - final_df["total_liabilities"]) / final_df[
        "shares"
    ]
    final_df["tpbr"] = final_df["tbps"] / final_df["price"]

    final_df["ctr"] = final_df["cbps"] / final_df["tbps"]

    final_df["npr"] = final_df["net_profit"] / final_df["raw_profit"]
    final_df["opr"] = final_df["operation_profit"] / final_df["raw_profit"]
    final_df = final_df.loc[:, ["stock_code", "cpbr", "tpbr", "ctr", "npr", "opr", "profit"]]
    final_df.set_index("stock_code", inplace=True)
    final_df = final_df[~(np.isinf(final_df).sum(axis=1).astype(bool))]
    final_df.dropna(inplace=True)
    return final_df

In [16]:
final_0 = append_factors(final_0)
final_1 = append_factors(final_1)
final_2 = append_factors(final_2)

In [17]:
using = set(final_0.index) and set(final_1.index) and set(final_2.index)

final_0 = final_0[final_0.index.isin(using)].copy()
final_1 = final_1[final_1.index.isin(using)].copy()
final_2 = final_2[final_2.index.isin(using)].copy()

In [18]:
final_0.to_csv("./example_0.csv")
final_1.to_csv("./example_1.csv")
final_2.to_csv("./example_2.csv")